In [66]:
# install needed stuff
!pip install tensorflow_decision_forests

In [67]:
# imports
import pandas as pd
import numpy as np
import time
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import tensorflow_decision_forests as tfdf

# seed
np.random.seed(0)

## Note for implementation
- Model: We apply Random Forests classifier implemented using scikit- learn toolkit16 and 100 decision tree estimators.
- 5 fold cross validation (quite special, detail in paper)
- Dataset has a 4 bands separation according to account's follower count

In [68]:
# where data is located
!ls ./drive/MyDrive/Thesis\ Workspace/Notebooks/data/set-2
DATA_PATH = "./drive/MyDrive/Thesis Workspace/Notebooks/data/sb14-set2/"

tweets	users


In [69]:
# load user data
df_bot = pd.read_csv(DATA_PATH + 'bot.csv')
df_naive = pd.read_csv(DATA_PATH + 'naive.csv')
print('Shapes', df_bot.shape, df_naive.shape)


Shapes (4912, 12) (3394, 12)


In [70]:
df_bot['label'] = 1
df_naive['label'] = 0
df = pd.concat([df_bot, df_naive], ignore_index=True)
print(df['label'].unique())
df.head()

[1 0]


,statuses_count,favourites_count,listed_count,age,tweet_frequency,user_id,favorite_received,favorite_received_ratio,retweet_received,retweet_received_ratio,url_count,activity_source_count,label
0,1299,1,0,6.995963,185.678511,24858289,8,0.003177,11,0.004369,2038,34,1
1,18665,16358,110,6.904962,2703.128629,33212890,31586,9.858302,37519,11.710050,1073,4,1
2,22987,14,6,6.839143,3361.093391,39773427,109,0.033893,477172,148.374378,2025,12,1
3,7975,11,2,6.666963,1196.196831,57007623,46,0.014237,171137,52.967193,479,8,1
4,20218,162,8,6.608517,3059.385303,63258466,834,0.260544,100333,31.344267,423,14,1


In [71]:
# train test split
train, val = train_test_split(df, train_size=0.8)
print(train.shape, val.shape)

(6644, 13) (1662, 13)


In [73]:
# Convert the dataset into a TensorFlow dataset.
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="label")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(val, label="label")

# Train a Random Forest model.
model = tfdf.keras.RandomForestModel()
model.fit(train_ds)

104/104 [==============================] - 3s 28ms/step


In [74]:
# Summary of the model structure.
model.summary()

Model: "random_forest_model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (12):
	activity_source_count
	age
	favorite_received
	favorite_received_ratio
	favourites_count
	listed_count
	retweet_received
	retweet_received_ratio
	statuses_count
	tweet_frequency
	url_count
	user_id

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.                 "__LABEL"  7.331854 ################
    2.            "listed_count"  6.672851 ##############
    3.          "statuses_count"  6.146248 ############
    4.                 "user_id"  6.074903 ############
    5.                     "age"  6.018574 ############
    6. "favorite_received_ratio"  5.998170 ############
    7.         "tweet_frequency"  5.75

### Note
1. No crossvalidation needed cause the model document states that:

"If true, compute the Out-of-bag evaluation (then available in the summary and model inspector). This evaluation is a cheap alternative to cross-validation evaluation. Default: True."

2. No data band separation since our data is so small

In [75]:
# Evaluate the model.
model.compile(metrics=['accuracy','AUC','Recall','Precision'])
print(model.evaluate(test_ds,return_dict=True))

26/26 [==============================] - 1s 9ms/step - loss: 0.0000e+00 - accuracy: 0.9916 - auc: 0.9970 - recall: 0.9899 - precision: 0.9960
{'loss': 0.0, 'accuracy': 0.9915764331817627, 'auc': 0.996954083442688, 'recall': 0.9899497628211975, 'precision': 0.9959555268287659}


In [76]:

pred = model.predict(test_ds) > 0.5
print(classification_report(pred, val['label'], digits=4))
print('ROC acc: ',roc_auc_score(pred, val['label']))


              precision    recall  f1-score   support

       False     0.9940    0.9851    0.9896       673
        True     0.9899    0.9960    0.9929       989

    accuracy                         0.9916      1662
   macro avg     0.9920    0.9905    0.9912      1662
weighted avg     0.9916    0.9916    0.9916      1662

ROC acc:  0.9905483348031918
